# Mitigation in Stable Diffusion Model
## Gender-Race detection

This code has to be run for each experiment, i.e each configuration of the fine-tuning method. This code is made to run the detection (gender and race) for a main folder containing 9 inside folders corresponding to the different prompts. In this case is set to run the detection model for the images generated by Dreambooth_constant2 (constant scheduler, 20 class images and 100 training steps).

You just have to change the folder path to run it for different configurations

Remember to set up a GPU before running the code!

In [ ]:
!nvidia-smi

Wed Aug  2 16:00:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install deepface
!pip install face_recognition
!pip install opencv-python
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 780.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=7592843fffc9c832ad04737b7f12705fcfae900c172cf6c7901b91c3a1489e6e
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566169 sha256=7fe71b7bf1ac2876a50f6aa432cfcfe4617ae5b700d7e7ce91d6300943bb79de
  Stored in directory: /root/.cache/pip/wh

If you're runing it in GoocleColab you need to set the connection to Google drive content by decomenting the following cell

In [ ]:
#from google.colab import drive
#import sys
#drive.mount('/content/drive')
#sys.path.insert(0,"/content/drive/MyDrive/Master/")

Mounted at /content/drive


In [ ]:
#Identify gender and ethnicty
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
import math
import csv
from deepface import DeepFace
import face_recognition
import itertools
import os
from utils import *


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


Make sure to set the right path if neccesary. If you're runing it in GoogleColab, set the path to the folder

This two paths would correspond to

* folder_drive: folder where the images to analyse are hosted : This folder has to have the following folders on the inside, each of them with the images corresponding to the name
    ['ceo', 'executive', 'plumbing', 'doctor', 'nice', 'nurse', 'criminal','delivery','entrepreneur']
* csv_path: path to the csv with the results

In [ ]:
folder_drive =  "dreambooth_results/Dreambooth_constant2"#"/content/drive/MyDrive/Master/dreambooth_results/Dreambooth_constant2"
csv_path = "dreambooth_results/dream_detect_constant_20.csv"#"/content/drive/MyDrive/Master/dreambooth_results/dream_detect_constant_20.csv"

## Models for gender and ethnicity

Run one time for each configuration folder. Define the
* Folder path with the results of the generated pictures for each prompt
* Destination csv filename

In [ ]:
%%time
with open(csv_path, 'w') as result:
    writer = csv.DictWriter(result, ['prompt', 'path', 'gender', 'age', 'race'])
    writer.writeheader()

    for folder in ['ceo', 'executive', 'plumbing', 'doctor', 'nice', 'nurse', 'criminal','delivery','entrepreneur']:
        # Generate images for each prompt
        for image in os.listdir(folder_drive + '/' + folder):
            image_path = folder_drive + '/' + folder + '/' + image
            #Try to detect age/gender
            try:
                output, gender, age = age_gender_detector(image_path)

                new_dict = {'prompt' : folder, 'path' : image_path,
                            'gender' : gender, 'age' : age, 'race' : 'race'}

            except:
                print(f'failed age/gender detection for {image_path}')
                new_dict = {'prompt' : folder, 'path' : image_path,
                            'gender' : '--', 'age' : '--', 'race' : '--'}

            #Try to detect race
            try:
                ethnicity = detect_ethnicity(image_path)
                dom_race = ethnicity[0]['dominant_race']
                new_dict['race'] = dom_race
            except:
                print(f'failed ethnicity detection for {image_path}')

            writer.writerow(new_dict)

Gender: Male, conf: 1.000
Age: (25-32), conf: 0.996


Action: race: 100%|██████████| 1/1 [00:00<00:00,  7.97it/s]


Ethnicity prediction: [{'race': {'asian': 0.09878454729914665, 'indian': 0.6103041581809521, 'black': 0.004088716741534881, 'white': 64.60119485855103, 'middle eastern': 20.950661599636078, 'latino hispanic': 13.734965026378632}, 'dominant_race': 'white', 'region': {'x': 0, 'y': 0, 'w': 108, 'h': 107}}]


Action: race: 100%|██████████| 1/1 [00:00<00:00,  8.27it/s]


Ethnicity prediction: [{'race': {'asian': 5.862726620598234, 'indian': 0.40240616142190466, 'black': 0.5747092764651344, 'white': 77.78030501486688, 'middle eastern': 3.412958065447692, 'latino hispanic': 11.966896025353433}, 'dominant_race': 'white', 'region': {'x': 0, 'y': 0, 'w': 36, 'h': 36}}]
Gender: Female, conf: 0.951
Age: (8-12), conf: 0.563


...

Ethnicity prediction: [{'race': {'asian': 0.28794314712285995, 'indian': 1.8032198771834373, 'black': 0.10250795166939497, 'white': 64.99239206314087, 'middle eastern': 24.9924436211586, 'latino hispanic': 7.821493595838547}, 'dominant_race': 'white', 'region': {'x': 0, 'y': 0, 'w': 52, 'h': 52}}]
Gender: Female, conf: 1.000
Age: (25-32), conf: 0.999


Action: race: 100%|██████████| 1/1 [00:00<00:00,  7.19it/s]


Ethnicity prediction: [{'race': {'asian': 4.310934130009063, 'indian': 3.9140673824588883, 'black': 0.3788135240264847, 'white': 29.845835557331775, 'middle eastern': 7.417784328328629, 'latino hispanic': 54.13256610230005}, 'dominant_race': 'latino hispanic', 'region': {'x': 0, 'y': 0, 'w': 90, 'h': 89}}]
Gender: Male, conf: 1.000
Age: (25-32), conf: 0.761


Action: race: 100%|██████████| 1/1 [00:00<00:00,  4.85it/s]


Ethnicity prediction: [{'race': {'asian': 4.516832157969475, 'indian': 6.43264576792717, 'black': 79.01306748390198, 'white': 0.6827265024185181, 'middle eastern': 0.5814190953969955, 'latino hispanic': 8.773311972618103}, 'dominant_race': 'black', 'region': {'x': 0, 'y': 0, 'w': 75, 'h': 74}}]
failed age/gender detection for /content/drive/MyDrive/Master/dreambooth_results/Dreambooth_constant2/entrepreneur/profile_99_constant2.png
failed ethnicity detection for /content/drive/MyDrive/Master/dreambooth_results/Dreambooth_constant2/entrepreneur/profile_99_constant2.png
CPU times: user 7min 58s, sys: 13.8 s, total: 8min 12s
Wall time: 8min 52s
